In [8]:
import os 
import sys 
from functools import cache
from itertools import product

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils import ddf, remove_prefix, load_subgraph

True

In [3]:
sg, bs = load_subgraph()

## Siloed Asset Breakdown 

In [3]:
from collections import OrderedDict

# See UI logic in this directory for reference 
# https://github.com/BeanstalkFarms/Beanstalk-UI/tree/master/src/components/Analytics/Silo
beanstalk_addr = '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5'
token_addrs = OrderedDict(dict(
    bean='0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab'.lower(), 
    bean_3crv='0xc9C32cd16Bf7eFB85Ff14e0c8603cc90F6F2eE49'.lower(), 
    ur_bean='0x1BEA0050E63e05FBb5D8BA2f10cf5800B6224449'.lower(), 
    ur_bean_3crv='0x1BEA3CcD22F4EBd3d37d731BA31Eeca95713716D'.lower(), 
))
asset_ids = [f"{beanstalk_addr.lower()}-{token_addr.lower()}" for token_addr in token_addrs.values()]
queries = [
    bs.Query.siloAssetHourlySnapshots(
        first=10000, orderBy="season", orderDirection="asc",
        where={"season_gte": 6074, "siloAsset": aid}
    )
    for aid in asset_ids 
]
df_snaps_raw = pd.concat([
    sg.query_df(
        [
            q.id, 
            q.season, 
            q.timestamp, 
            q.totalDepositedAmount, 
            q.totalDepositedBDV, 
        ],
        pagination_strategy=ShallowStrategy
    )
    for q in queries
])

In [8]:

from collections import defaultdict


df_dep = df_snaps_raw.copy()
df_dep = remove_prefix(df_dep, 'siloAssetHourlySnapshots_').sort_values('timestamp').reset_index(drop=True)
df_dep['token'] = df_dep['id'].apply(lambda v: v.split('-')[1])
# df_dep = df_dep.groupby(['season', 'token']).last().reset_index()
# seasons = sorted(df_dep.season.unique())
# tokens = df_dep.token.unique()
# records = []
token_addr_inv = {v: k for k, v in token_addrs.items()}
# token_mult = OrderedDict(dict(
#     bean=1e-6, 
#     bean_3crv=1e-18, 
#     ur_bean=1e-6, 
#     ur_bean_3crv=1e-12, 
# ))
# last_value = defaultdict(float)
# for s, t in product(seasons, tokens):
#     metrics = {}
#     for m in ['totalDepositedAmount', 'totalDepositedBDV']: 
#         v = df_dep.loc[(df_dep.season == s) & (df_dep.token == t)][m]
#         if len(v): 
#             v = v.values[0]
#         else: 
#             v = last_value.get((t,m), 0)
#         v = v * token_mult[token_addr_inv[t]]
#         last_value[(t,m)] = v
#         metrics[m] = v
#     records.append({
#         "season": s, 
#         "token": t, 
#         **metrics 
#     })
# df_dep = pd.DataFrame(records)

df_dep['token_name'] = df_dep.token.apply(lambda v: token_addr_inv[v])

df_dep = df_dep.loc[df_dep.token_name == 'bean']
ddf(df_dep.tail(10))

,id,season,timestamp,totalDepositedAmount,totalDepositedBDV,token,token_name
1303,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6526,6526,1661430012,1042244896932,1073468740691,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1320,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6527,6527,1661435670,1042258981877,1073476969634,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1321,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6528,6528,1661439253,1042323471238,1073605948356,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1325,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6529,6529,1661442210,1045952378186,1080682771610,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1326,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6530,6530,1661445224,1045998329402,1077262268375,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1329,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6531,6531,1661448129,1046191666171,1077444807428,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1332,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6532,6532,1661453998,1046835912314,1078091298122,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1333,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6492,6492,1661454280,1046835912314,1078299467886,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1336,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6533,6533,1661456781,1052034716326,1083308065119,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1337,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6534,6534,1661460075,1053200024065,1085397673759,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean


In [6]:
df_dep = df_dep.loc[df_dep.token_name == 'bean']
alt.Chart(df_dep).mark_line().encode(
    x=alt.X("season:O"), 
    y=alt.Y("totalDepositedAmount:Q"), 
    color=alt.Color("token_name:N"), 
    tooltip=alt.Tooltip("totalDepositedAmount:Q")
).properties(width=700)

alt.Chart(...)

In [10]:
"0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5".lower()

'0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5'

In [11]:
"0x1BEA0050E63e05FBb5D8BA2f10cf5800B6224449".lower()

'0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449'

In [ ]:
# query SeasonalDepositedUnripeBeans {
#   siloAssetDailySnapshots(
#     where: {
#       siloAsset: "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449"
#     }
#     orderBy: season
#     orderDirection: desc
#     first: 10 
#   ) {
#     season
#     siloAsset {
#       totalDepositedAmount
#     	totalDepositedBDV
#     }
#   }
# }

## Beanstalk Credit Breakdown (Daily + Cumulative)

Credit 
- Silo 
  - emissions (in-progress)
- Barn 
  - sprouts rinsed / rinsable (done)
- Field 
  - pods harvested (done)
  - pods harvestable (done)
  
Debt
- Field
  - pods issued (done) 
- Barn 
  - sprouts

In [71]:
@cache
def query_rewards(refresh=None): 
    rewards = bs.Query.rewards(orderBy="blockNumber", orderDirection="asc", first=10000)
    df_rewards = sg.query_df(
        [
            rewards.season, 
            rewards.toFertilizer, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df_rewards 

In [102]:
# fertilizer emissions (incomplete season axis)
df_rewards = query_rewards().copy()
df_rewards = remove_prefix(df_rewards, 'rewards_')
df_rewards['sprouts_rinsed_daily'] = df_rewards.toFertilizer.astype(float) / 1e6
df_rewards['sprouts_rinsed_cumulative'] = df_rewards.sprouts_rinsed_daily.cumsum()
df_rewards = df_rewards.drop(columns=["toFertilizer"])
df_rewards.tail()

,season,sprouts_rinsed_daily,sprouts_rinsed_cumulative
366,6537,0.000000,2.094306e+06
367,6538,120.490265,2.094426e+06
368,6539,120.490265,2.094547e+06
369,6540,68.851580,2.094616e+06
370,6541,34.425790,2.094650e+06


In [103]:
@cache 
def query_field(refresh=None) -> pd.DataFrame: 
    field_snaps = bs.Query.fieldDailySnapshots(
        orderBy="season", 
        orderDirection="asc", 
        first=10000, 
        where={"field": "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5"}
    )
    df_field = sg.query_df(
        [
            field_snaps.season, 
            field_snaps.newHarvestedPods, 
            field_snaps.newHarvestablePods, 
            field_snaps.podIndex, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df_field 

In [118]:
df_field = query_field(refresh=3).copy()
df_field = remove_prefix(df_field, "fieldDailySnapshots_")
df_field['pods_harvestable_daily'] = (df_field.newHarvestablePods / 10**6)
df_field['pods_harvestable_cumulative'] = df_field.pods_harvestable_daily.cumsum()
df_field['pods_issued_cumulative'] = df_field.podIndex / 10**6
df_field['pods_issued_daily'] = df_field.pods_issued_cumulative - df_field.pods_issued_cumulative.shift(1).fillna(0)
df_field['pods_harvested_daily'] = df_field.newHarvestedPods / 10**6
df_field['pods_harvested_cumulative'] = df_field.pods_harvested_daily.cumsum()
df_field = df_field.drop(columns=['podIndex', 'newHarvestedPods', 'newHarvestablePods'])
df_field.tail()

,season,pods_harvestable_daily,pods_harvestable_cumulative,pods_issued_cumulative,pods_issued_daily,pods_harvested_daily,pods_harvested_cumulative
273,6465,2132.005212,5.753024e+07,8.210746e+08,1066.001986,0.000000,5.752809e+07
274,6489,1499.181633,5.753174e+07,8.210963e+08,21662.109986,0.000000,5.752809e+07
275,6513,3181.584188,5.753492e+07,8.210979e+08,1590.791450,5715.895364,5.753380e+07
276,6537,3188.827863,5.753811e+07,8.210996e+08,1716.976045,0.000000,5.753380e+07
277,6541,386.968521,5.753850e+07,8.210998e+08,193.484147,0.000000,5.753380e+07


In [109]:
@cache 
def query_silo(refresh=None) -> pd.DataFrame: 
    silo_snaps = bs.Query.siloDailySnapshots(
        orderBy="season", 
        orderDirection="asc", 
        first=10000, 
        where={"silo": "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5"}
    )
    df = sg.query_df(
        [
            silo_snaps.season, 
            silo_snaps.dailyBeanMints, 
            silo_snaps.totalBeanMints, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df 

In [110]:
df_silo = query_silo().copy()
print(len(df_silo))
df_silo = remove_prefix(df_silo, "siloDailySnapshots_")
df_silo['silo_emissions_daily'] = df_silo.dailyBeanMints / 10**6
df_silo['silo_emissions_cumulative'] = df_silo.totalBeanMints / 10**6
df_silo = df_silo.drop(columns=['dailyBeanMints', 'totalBeanMints'])
df_silo.tail()

23


,season,silo_emissions_daily,silo_emissions_cumulative
18,6465,2132.005223,2.091284e+06
19,6489,1499.181643,2.092783e+06
20,6513,3181.584200,2.095965e+06
21,6520,3188.827871,2.099153e+06
22,6542,426.510736,2.099580e+06


In [111]:
@cache 
def query_seasons(refresh=None) -> pd.DataFrame: 
    seasons = bs.Query.seasons(
        first=10000, where={"season_gte": 6074}, orderBy="season", orderDirection="asc"
    )
    df = sg.query_df([
        seasons.season, 
        seasons.timestamp, 
    ], pagination_strategy=ShallowStrategy)
    df = remove_prefix(df, 'seasons_')
    return df 

In [129]:
df_szns = query_seasons()
df_szns['timestamp'] = pd.to_datetime(df_szns.timestamp, unit='s')
df_szns.head()

,season,timestamp
0,6074,2022-04-17 12:00:10
1,6075,2022-08-06 17:00:14
2,6076,2022-08-06 18:02:11
3,6077,2022-08-06 19:01:04
4,6078,2022-08-06 20:00:02


In [138]:
df = df_szns.merge(
    df_rewards, how='left', on='season'
).merge(
    df_field, how='left', on='season'
).merge(
    df_silo, how='left', on='season'
)
df = df.loc[df.season >= 6075]
df = df.ffill().fillna(0) # Not technically correct but close enough 
# df_debt = df['season', 'pods_issued_cumulative']
df_credit = df[[
    'timestamp', 
    'sprouts_rinsed_cumulative', 
    'pods_harvestable_cumulative',
    'pods_harvested_cumulative', 
    'silo_emissions_cumulative',
]].melt(
    id_vars=['timestamp'], 
    value_vars=[
        'sprouts_rinsed_cumulative', 
        'pods_harvestable_cumulative',
        'pods_harvested_cumulative', 
        'silo_emissions_cumulative',
    ]
).sort_values("timestamp").reset_index(drop=True)
df_credit.head(20)
# credit = alt.Chart(df_credit).mark_area().encode(
#     x="yearmonthdate(timestamp):T", 
#     y="value:Q", 
#     color="variable:N"
# ).properties(width=750)
# credit

,timestamp,variable,value
0,2022-08-06 17:00:14,sprouts_rinsed_cumulative,0.000000
1,2022-08-06 17:00:14,silo_emissions_cumulative,0.000000
2,2022-08-06 17:00:14,pods_harvested_cumulative,0.000000
3,2022-08-06 17:00:14,pods_harvestable_cumulative,0.000000
4,2022-08-06 18:02:11,silo_emissions_cumulative,0.000000
5,2022-08-06 18:02:11,pods_harvestable_cumulative,0.000000
6,2022-08-06 18:02:11,pods_harvested_cumulative,0.000000
7,2022-08-06 18:02:11,sprouts_rinsed_cumulative,2266.788451
8,2022-08-06 19:01:04,sprouts_rinsed_cumulative,6824.618896
9,2022-08-06 19:01:04,pods_harvested_cumulative,0.000000
